In [0]:
%sql
MERGE INTO gold.customers AS target
USING (
    SELECT 
        c.customer_id,
        c.name,
        c.email,
        c.usd_balance_original,  -- Mantendo saldo original de USD
        c.btc_balance_original,  -- Mantendo saldo original de BTC
        -- Calculando saldo atualizado de BTC considerando compras e vendas
        c.btc_balance_original + COALESCE(SUM(
            CASE 
                WHEN t.transaction_type = 'compra' THEN t.btc_amount  -- Se comprou, adiciona BTC
                WHEN t.transaction_type = 'venda' THEN -t.btc_amount  -- Se vendeu, reduz BTC
                ELSE 0
            END
        ), 0) AS btc_balance_final,
        COUNT(t.transaction_id) AS total_transactions,
        -- Calculando total de USD gasto somando transaction_value_in_usd com usd_balance_original
        c.usd_balance_original + COALESCE(SUM(t.transaction_value_in_usd), 0) AS usd_balance_final
    FROM silver.customers c
    LEFT JOIN gold.transaction t ON c.customer_id = t.customer_id
    GROUP BY c.customer_id, c.name, c.email, c.usd_balance_original, c.btc_balance_original
) AS source
ON target.customer_id = source.customer_id
WHEN MATCHED THEN
  UPDATE SET
    target.name = source.name,
    target.email = source.email,
    target.usd_balance_original = source.usd_balance_original,
    target.btc_balance_original = source.btc_balance_original,
    target.btc_balance_final = source.btc_balance_final,
    target.total_transactions = source.total_transactions,
    target.usd_balance_final = source.usd_balance_final
WHEN NOT MATCHED
  THEN INSERT (customer_id, name, email, usd_balance_original, btc_balance_original, btc_balance_final, total_transactions, usd_balance_final)
  VALUES (source.customer_id, source.name, source.email, source.usd_balance_original, source.btc_balance_original, source.btc_balance_final, source.total_transactions, source.usd_balance_final);